In [1]:
!cat /proc/meminfo

MemTotal:       32949048 kB
MemFree:        31027120 kB
MemAvailable:   31769548 kB
Buffers:          125700 kB
Cached:           942456 kB
SwapCached:            0 kB
Active:          1173424 kB
Inactive:         569140 kB
Active(anon):     674500 kB
Inactive(anon):       48 kB
Active(file):     498924 kB
Inactive(file):   569092 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:             57864 kB
Writeback:             0 kB
AnonPages:        674780 kB
Mapped:           123704 kB
Shmem:                60 kB
Slab:              98072 kB
SReclaimable:      75644 kB
SUnreclaim:        22428 kB
KernelStack:        3872 kB
PageTables:         6060 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    16474524 kB
Committed_AS:    1744784 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

In [2]:
!pip install xgboost msgpack

    100% |████████████████████████████████| 15.8MB 3.3MB/s eta 0:00:01
    100% |████████████████████████████████| 317kB 41.2MB/s ta 0:00:01
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import gc
import pickle

from sklearn import preprocessing as prp
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import (accuracy_score, r2_score, confusion_matrix,
f1_score, recall_score, precision_score)

import os
import boto3
import re

import xgboost as xgb

# ruta de trabajo en s3
bucket = 'boi-banregio'
target = 'BMI'
feat_part_number = 40

# s3 resource
s3_bucket_resource = boto3.resource('s3').Bucket(bucket)

In [4]:
from itertools import chain, zip_longest

def predict(probs, threshold):
    preds = []
    for prob in probs:
        if prob > threshold:
            preds.append(1)
        else:
            preds.append(0)
    return np.array(preds)

# fval for hyperparameters
def val_func(pred_probs, dmat):
    thrs = [0.25,0.27,0.3,0.32,0.34]
    ret = []
    for thr in thrs:
        preds = predict(pred_probs, thr)
        score = f1_score(preds, dmat.get_float_info('label'))
        ret.append(('f1_score_thr{}'.format(thr), score))
    return ret

def pretty_table(matrix):
    matrix_aux = chain.from_iterable(
        zip_longest(
            *(x.splitlines() for x in y),
            fillvalue='')
        for y in [[str(e) for e in row] for row in matrix])

    s = [[str(e) for e in row] for row in matrix_aux]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print('\n'.join(table))
    
def get_xgb_feat_importances(clf):
    if isinstance(clf, xgb.XGBModel):
        # clf has been created by calling
        # xgb.XGBClassifier.fit() or xgb.XGBRegressor().fit()
        fscore = clf.booster().get_fscore()
    else:
        # clf has been created by calling xgb.train.
        # Thus, clf is an instance of xgb.Booster.
        fscore = clf.get_fscore()

    feat_importances = []
    for ft, score in fscore.items():
        feat_importances.append({'Feature': ft, 'Importance': score})
    feat_importances = pd.DataFrame(feat_importances)
    feat_importances = feat_importances.sort_values(
        by='Importance', ascending=False).reset_index(drop=True)
    # Divide the importances by the sum of all importances
    # to get relative importances. By using relative importances
    # the sum of all importances will equal to 1, i.e.,
    # np.sum(feat_importances['importance']) == 1
    feat_importances['Importance'] /= feat_importances['Importance'].sum()
    # Print the most important features and their importances
    print(feat_importances.head())
    return feat_importances

In [12]:
# Aquí colocamos la ruta de nuestro directorio, el nombre de nuestro archivo y si tiene o no cabecera
prefix = 'datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION'

In [13]:
target_file_name = "TARGETS.csv"
target_file_key = "{}/{}".format(prefix, target_file_name)
target_obj = s3_bucket_resource.Object(target_file_key).get()
target_data = pd.read_csv(io.BytesIO(target_obj['Body'].read())).loc[:,[target]]

In [14]:
# model_features_file_name = "MODEL_DATASET/MODEL_FEATURES.csv"
model_features_file_name = "MODEL_DATASET/MODEL_FEATURES+EXTRAS.csv"
model_features_file_key = "{}/{}".format(prefix, model_features_file_name)
model_features_obj = s3_bucket_resource.Object(model_features_file_key).get()
model_features_data = pd.read_csv(io.BytesIO(model_features_obj['Body'].read()))

In [15]:
np.random.seed(0)
rand_split = np.random.rand(len(model_features_data))
train_list = rand_split < 0.7
val_list = (rand_split >= 0.6) & (rand_split < 0.8)
test_list = rand_split >= 0.7

feature_data_train = model_features_data[train_list]
feature_data_val = model_features_data[val_list]
feature_data_test = model_features_data[test_list]

target_data_train = target_data[train_list]
target_data_val = target_data[val_list]
target_data_test = target_data[test_list]

In [16]:
train_y = target_data_train[target].values
val_y = target_data_val[target].values
test_y = target_data_test[target].values

train_X = feature_data_train.loc[:,:].values
val_X = feature_data_val.loc[:,:].values
test_X = feature_data_test.loc[:,:].values

dtrain = xgb.DMatrix(
    train_X, 
    label=train_y, 
    feature_names=feature_data_train.columns
)
dval = xgb.DMatrix(
    val_X, 
    label=val_y, 
    feature_names=feature_data_val.columns
)
dtest = xgb.DMatrix(
    test_X, 
    label=test_y, 
    feature_names=feature_data_test.columns
)

learning_rate =0.1,
n_estimators=200,
max_depth=12,
min_child_weight=19.50392156038251,
gamma= 0.001,
subsample=1,  # 0.8
colsample_bytree=1,  # 0.4
objective= 'binary:logistic',
#nthread=4,
reg_lambda=10,  # 17
max_delta_step= 50,  # 100
scale_pos_weight=24,  # 100
early_stopping_rounds = 100,
stratified = True,
seed=1

params = {
    'objective': 'binary:logistic', 
    'eta':0.1, 
    'max_depth': 12, 
    'eval_metric': 'auc',
    'max_delta_step': 10,
    'min_child_weight': 19.50392156038251,
    'gamma': 0.001,
    'subsample': 1,  # 0.8
    'colsample_bytree': 1,  # 0.4
    # nthread=4,
    'reg_lambda': 10,  # 17
    'max_delta_step': 50,  # 100
    'scale_pos_weight': 24,  # 100
    'early_stopping_rounds': 100,
    'stratified': True,
    'seed': 1
}
num_round = 100

In [ ]:
bst = pickle.load(open('models/pickles/cartera_vencida_version_hyertuning0.pckl.model', "rb"))

In [24]:
bst.__dict__

{'feature_names': ['AVG_R_BAL_BY_CRED_LIM_12M',
  'DEUDA_TOT',
  'MIN_R_BAL_BY_MAX_BAL_03M',
  'MAX_D_CRED_LIM_TO_BAL_03M',
  'AVG_D_CRED_LIM_TO_BAL_06M',
  'AVG_PND_MENOS_R_BAL_BY_MAX_BAL_06M',
  'AVG_R_BAL_BY_MAX_BAL_12M',
  'AVG_PND_MENOS_BAL_12M',
  'BK12_MAX_CREDIT_AMT',
  'AVG_PND_MAS_BAL_12M',
  'MAX_VIGENTEC_BRG_R_OP_ACC_03M',
  'AVG_PND_MENOS_VIGENTEH_06M',
  'MIN_SALINI_OP_ACC',
  'AVG_VIGENTEC_OP_ACC_06M',
  'AVG_VIGENTEC_BRG_OP_ACC',
  'AVG_MAX_DIAS_PARA_PAGO_U06M',
  'AVG_AVG_HI_CALCAR_L_12M',
  'AVG_PCT_VENC29_12M',
  'AVG_AVG_HI_CALCAR_CEIL_12M',
  'AVG_PND_MENOS_MAX_HI_CALCAR_L_03M',
  'MAX_AVG2_VP_MES_U06M',
  'AVG_BAL_12M',
  'AVG_PND_MENOS_AVG_HI_CALCAR_FLOOR_06M',
  'PCT_BK_CS_CL_ACC',
  'PCT_BK_CS_OP_ACC',
  'AVG_AVG2_VP_MES_U06M_2',
  'MIN_SALINI_BRG_CL_ACC',
  'MIN_PCT_VIGENTEH_12M',
  'AVG_AVG1_VP_PER_U06M_2',
  'AVG2_VP_MES_2',
  'AVG_AVG1_VP_PER_U03M_2',
  'AVG_AVG2_VP_MES_U03M_2',
  'AVG_VIGENTEC_BK_R_OP_ACC_06M',
  'AVG_VIGENTEC_BRG_CS_OP_ACC',
  'MAX_SALINI

In [ ]:
# Trianing
version = 0
params = {
    'objective': 'binary:logistic', 
    'eta': 0.3, 
    'max_depth': 10,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric': 'auc',
    'seed': 1,
    'lambda': 100,
    'alpha': 50
}

num_round = 2000

evals = [(dtest, 'eval')]

#bst = xgb.train(params, dtrain, num_round, evals, feval=val_func)
bst = xgb.train(params, dtrain, num_round, evals, feval=val_func, xgb_model=bst)
#hist = xgb.cv(params, dtrain, num_round, nfold=5, feval=val_func)


# pickle.dump(bst, open('models/pickles/cartera_vencida_version{}.pckl.model'.format(version),"wb"))
# feat_imp.to_csv('models/feature_importances_version{}.csv'.format(version), index=False)

[16:06:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=5
[0]	eval-auc:0.983851	eval-f1_score_thr0.25:0.743112	eval-f1_score_thr0.27:0.744507	eval-f1_score_thr0.3:0.747129	eval-f1_score_thr0.32:0.746441	eval-f1_score_thr0.34:0.744142
[16:06:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
[1]	eval-auc:0.983842	eval-f1_score_thr0.25:0.74378	eval-f1_score_thr0.27:0.744335	eval-f1_score_thr0.3:0.746828	eval-f1_score_thr0.32:0.746133	eval-f1_score_thr0.34:0.744562
[16:06:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
[2]	eval-auc:0.983862	eval-f1_score_thr0.25:0.743634	eval-f1_score_thr0.27:0.744919	eval-f1_score_thr0.3:0.747056	eval-f1_score_thr0.32:0.745102	eval-f1_score_thr0.34:0.744533
[16:06:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned 

[27]	eval-auc:0.983967	eval-f1_score_thr0.25:0.744106	eval-f1_score_thr0.27:0.74742	eval-f1_score_thr0.3:0.747403	eval-f1_score_thr0.32:0.746604	eval-f1_score_thr0.34:0.744851
[16:07:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=5
[28]	eval-auc:0.983977	eval-f1_score_thr0.25:0.744491	eval-f1_score_thr0.27:0.746667	eval-f1_score_thr0.3:0.747332	eval-f1_score_thr0.32:0.748046	eval-f1_score_thr0.34:0.74546
[16:07:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=5
[29]	eval-auc:0.984	eval-f1_score_thr0.25:0.744584	eval-f1_score_thr0.27:0.746397	eval-f1_score_thr0.3:0.747182	eval-f1_score_thr0.32:0.747712	eval-f1_score_thr0.34:0.745855
[16:07:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=4
[30]	eval-auc:0.984	eval-f1_score_thr0.25:0.744031	eval-f1_score_thr0.27:0.746638	eval-f1_score_thr0.3:0.

[16:08:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=5
[55]	eval-auc:0.984175	eval-f1_score_thr0.25:0.746175	eval-f1_score_thr0.27:0.747152	eval-f1_score_thr0.3:0.749445	eval-f1_score_thr0.32:0.749588	eval-f1_score_thr0.34:0.74551
[16:08:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[56]	eval-auc:0.984182	eval-f1_score_thr0.25:0.746385	eval-f1_score_thr0.27:0.747955	eval-f1_score_thr0.3:0.750177	eval-f1_score_thr0.32:0.749974	eval-f1_score_thr0.34:0.744958
[16:08:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=6
[57]	eval-auc:0.984171	eval-f1_score_thr0.25:0.746317	eval-f1_score_thr0.27:0.747443	eval-f1_score_thr0.3:0.750076	eval-f1_score_thr0.32:0.74964	eval-f1_score_thr0.34:0.745115
[16:08:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned

[82]	eval-auc:0.984332	eval-f1_score_thr0.25:0.748288	eval-f1_score_thr0.27:0.749148	eval-f1_score_thr0.3:0.751713	eval-f1_score_thr0.32:0.749383	eval-f1_score_thr0.34:0.747403
[16:10:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=5
[83]	eval-auc:0.984348	eval-f1_score_thr0.25:0.748049	eval-f1_score_thr0.27:0.74871	eval-f1_score_thr0.3:0.752519	eval-f1_score_thr0.32:0.750514	eval-f1_score_thr0.34:0.747089
[16:10:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=4
[84]	eval-auc:0.984361	eval-f1_score_thr0.25:0.748286	eval-f1_score_thr0.27:0.749245	eval-f1_score_thr0.3:0.752044	eval-f1_score_thr0.32:0.751055	eval-f1_score_thr0.34:0.74735
[16:10:12] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=5
[85]	eval-auc:0.984372	eval-f1_score_thr0.25:0.747621	eval-f1_score_thr0.27:0.749635	eval-f1_score_thr

[16:11:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=4
[110]	eval-auc:0.984465	eval-f1_score_thr0.25:0.748361	eval-f1_score_thr0.27:0.749126	eval-f1_score_thr0.3:0.754052	eval-f1_score_thr0.32:0.751543	eval-f1_score_thr0.34:0.748271
[16:11:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=5
[111]	eval-auc:0.984481	eval-f1_score_thr0.25:0.748409	eval-f1_score_thr0.27:0.749029	eval-f1_score_thr0.3:0.754428	eval-f1_score_thr0.32:0.751645	eval-f1_score_thr0.34:0.748429
[16:11:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=6
[112]	eval-auc:0.984488	eval-f1_score_thr0.25:0.748216	eval-f1_score_thr0.27:0.749586	eval-f1_score_thr0.3:0.754964	eval-f1_score_thr0.32:0.75139	eval-f1_score_thr0.34:0.748374
[16:11:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 p

...

[16:55:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[770]	eval-auc:0.986756	eval-f1_score_thr0.25:0.767528	eval-f1_score_thr0.27:0.771753	eval-f1_score_thr0.3:0.772673	eval-f1_score_thr0.32:0.772458	eval-f1_score_thr0.34:0.772628
[16:55:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=4
[771]	eval-auc:0.986758	eval-f1_score_thr0.25:0.767393	eval-f1_score_thr0.27:0.771981	eval-f1_score_thr0.3:0.772444	eval-f1_score_thr0.32:0.772145	eval-f1_score_thr0.34:0.771783
[16:55:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=4
[772]	eval-auc:0.986773	eval-f1_score_thr0.25:0.767334	eval-f1_score_thr0.27:0.77225	eval-f1_score_thr0.3:0.771736	eval-f1_score_thr0.32:0.772099	eval-f1_score_thr0.34:0.771703
[16:55:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 prun

[797]	eval-auc:0.986844	eval-f1_score_thr0.25:0.768312	eval-f1_score_thr0.27:0.772502	eval-f1_score_thr0.3:0.77339	eval-f1_score_thr0.32:0.772574	eval-f1_score_thr0.34:0.772841
[16:56:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=4
[798]	eval-auc:0.986827	eval-f1_score_thr0.25:0.768532	eval-f1_score_thr0.27:0.772489	eval-f1_score_thr0.3:0.773015	eval-f1_score_thr0.32:0.772951	eval-f1_score_thr0.34:0.773078
[16:56:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=4
[799]	eval-auc:0.986838	eval-f1_score_thr0.25:0.768341	eval-f1_score_thr0.27:0.772851	eval-f1_score_thr0.3:0.772942	eval-f1_score_thr0.32:0.773095	eval-f1_score_thr0.34:0.773306
[16:56:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[800]	eval-auc:0.986838	eval-f1_score_thr0.25:0.768341	eval-f1_score_thr0.27:0.772851	eval-f1_score_

[16:58:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=4
[825]	eval-auc:0.986877	eval-f1_score_thr0.25:0.768212	eval-f1_score_thr0.27:0.77301	eval-f1_score_thr0.3:0.772618	eval-f1_score_thr0.32:0.774114	eval-f1_score_thr0.34:0.772372
[16:58:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
[826]	eval-auc:0.986886	eval-f1_score_thr0.25:0.768197	eval-f1_score_thr0.27:0.773567	eval-f1_score_thr0.3:0.77217	eval-f1_score_thr0.32:0.774055	eval-f1_score_thr0.34:0.773078
[16:58:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[827]	eval-auc:0.986886	eval-f1_score_thr0.25:0.768197	eval-f1_score_thr0.27:0.773567	eval-f1_score_thr0.3:0.77217	eval-f1_score_thr0.32:0.774055	eval-f1_score_thr0.34:0.773078
[16:58:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 prune

[852]	eval-auc:0.986911	eval-f1_score_thr0.25:0.768547	eval-f1_score_thr0.27:0.773178	eval-f1_score_thr0.3:0.773202	eval-f1_score_thr0.32:0.7735	eval-f1_score_thr0.34:0.772869
[17:00:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
[853]	eval-auc:0.986907	eval-f1_score_thr0.25:0.769012	eval-f1_score_thr0.27:0.772612	eval-f1_score_thr0.3:0.773234	eval-f1_score_thr0.32:0.773374	eval-f1_score_thr0.34:0.77303
[17:00:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[854]	eval-auc:0.986907	eval-f1_score_thr0.25:0.769012	eval-f1_score_thr0.27:0.772612	eval-f1_score_thr0.3:0.773234	eval-f1_score_thr0.32:0.773374	eval-f1_score_thr0.34:0.77303
[17:00:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[855]	eval-auc:0.986911	eval-f1_score_thr0.25:0.769274	eval-f1_score_thr0.27:0.772449	eval-f1_score_th

[17:01:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[880]	eval-auc:0.986947	eval-f1_score_thr0.25:0.769566	eval-f1_score_thr0.27:0.774438	eval-f1_score_thr0.3:0.774174	eval-f1_score_thr0.32:0.775079	eval-f1_score_thr0.34:0.773225
[17:01:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[881]	eval-auc:0.986955	eval-f1_score_thr0.25:0.769289	eval-f1_score_thr0.27:0.773811	eval-f1_score_thr0.3:0.774219	eval-f1_score_thr0.32:0.774828	eval-f1_score_thr0.34:0.774106
[17:01:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=4
[882]	eval-auc:0.986942	eval-f1_score_thr0.25:0.769858	eval-f1_score_thr0.27:0.773647	eval-f1_score_thr0.3:0.774887	eval-f1_score_thr0.32:0.774921	eval-f1_score_thr0.34:0.774362
[17:01:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pru

In [19]:
hist

,test-error-mean,test-error-std,test-f1_score_thr0.1-mean,test-f1_score_thr0.1-std,test-f1_score_thr0.2-mean,test-f1_score_thr0.2-std,test-f1_score_thr0.3-mean,test-f1_score_thr0.3-std,test-f1_score_thr0.4-mean,test-f1_score_thr0.4-std,...,train-f1_score_thr0.5-mean,train-f1_score_thr0.5-std,train-f1_score_thr0.6-mean,train-f1_score_thr0.6-std,train-f1_score_thr0.7-mean,train-f1_score_thr0.7-std,train-f1_score_thr0.8-mean,train-f1_score_thr0.8-std,train-f1_score_thr0.9-mean,train-f1_score_thr0.9-std
0,0.035961,0.000834,0.075496,0.000666,0.075496,0.000666,0.075496,0.000666,0.075496,0.000666,...,0.332240,0.020899,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.035424,0.000610,0.075496,0.000666,0.075496,0.000666,0.075496,0.000666,0.075496,0.000666,...,0.310475,0.009841,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.035490,0.000589,0.075496,0.000666,0.075496,0.000666,0.075496,0.000666,0.415176,0.004522,...,0.308834,0.012430,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.035279,0.000665,0.075496,0.000666,0.075496,0.000666,0.075496,0.000666,0.446209,0.009986,...,0.308344,0.008383,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.035326,0.000730,0.075496,0.000666,0.075496,0.000666,0.102412,0.033377,0.445831,0.009132,...,0.304290,0.009119,0.013430,0.026646,0.000000,0.000000,0.0,0.0,0.0,0.0
5,0.035253,0.000691,0.075496,0.000666,0.075496,0.000666,0.338555,0.009278,0.442965,0.008047,...,0.305108,0.006554,0.106753,0.034738,0.000000,0.000000,0.0,0.0,0.0,0.0
6,0.035209,0.000612,0.075496,0.000666,0.075496,0.000666,0.411713,0.003437,0.435627,0.012153,...,0.309129,0.006701,0.143182,0.039043,0.000000,0.000000,0.0,0.0,0.0,0.0
7,0.035175,0.000753,0.075496,0.000666,0.075496,0.000666,0.438591,0.005167,0.430662,0.011916,...,0.311702,0.008578,0.172511,0.021525,0.000000,0.000000,0.0,0.0,0.0,0.0
8,0.035111,0.000664,0.075496,0.000666,0.075496,0.000666,0.449382,0.005878,0.425754,0.012352,...,0.314868,0.007443,0.188418,0.014548,0.000000,0.000000,0.0,0.0,0.0,0.0
9,0.035105,0.000775,0.075496,0.000666,0.257164,0.003647,0.454105,0.006705,0.420949,0.011235,...,0.313484,0.008341,0.202584,0.012987,0.000000,0.000000,0.0,0.0,0.0,0.0


In [ ]:
pickle.dump(bst, open('models/pickles/cartera_vencida_version{}.pckl.model'.format(version),"wb"))
feat_imp.to_csv('models/feature_importances_version{}.csv'.format(version), index=False)

In [ ]:
feat_imp = get_xgb_feat_importances(bst)

print('\n\n')
display(feat_imp)

In [26]:
f1_sc = 0
max_step = 0
thr_sample = (dtest, test_y)
_score_preds = bst.predict(thr_sample[0])
for thr_step in np.linspace(0, 1, 101):
    _preds = predict(_score_preds, thr_step)
    f1_sc_step = f1_score(thr_sample[1], _preds)
    print(thr_step, f1_sc_step)
    if f1_sc_step >= f1_sc:
        f1_sc = f1_sc_step
        max_step = thr_step
threshold = max_step
print('threshold', threshold)
print('max score', f1_sc)

0.0 0.0761418251065502
0.01 0.25410795136592557
0.02 0.34246052403262184
0.03 0.40759440851241396
0.04 0.4570456478593706
0.05 0.4975523382980939
0.06 0.5317737037453605
0.07 0.558395032242656
0.08 0.5834489718682982
0.09 0.6040250742329265
0.1 0.6233587681308861
0.11 0.6399658533115173
0.12 0.6550051447890637
0.13 0.6678774120317821
0.14 0.6805101882096749
0.15 0.689594777485869
0.16 0.6995918367346938
0.17 0.7068908122503329
0.18 0.7138853503184714
0.19 0.7194816414686824
0.2 0.7255298566528889
0.21 0.7302013422818792
0.22 0.7344119251045265
0.23 0.7388734995383196
0.24 0.7418781013013764
0.25 0.7432265424470007
0.26 0.7439130016360312
0.27 0.744792680552852
0.28 0.7450516986706056
0.29 0.7456445993031358
0.3 0.7466478475652787
0.31 0.7467373572593801
0.32 0.7462070389101042
0.33 0.7449139280125195
0.34 0.74405388339297
0.35000000000000003 0.7410865874363327
0.36 0.7395264116575592
0.37 0.7387874202961201
0.38 0.7365204103907445
0.39 0.7344334433443345
0.4 0.7315503273776495
0.410000

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [89]:
threshold = 0.17

In [27]:
# make predictions on datasets
train_pred = predict(bst.predict(dtrain), threshold)
val_pred = predict(bst.predict(dval), threshold)
test_pred = predict(bst.predict(dtest), threshold)

cm_train = confusion_matrix(train_y, train_pred)
cm_val = confusion_matrix(val_y, val_pred)
cm_test = confusion_matrix(test_y, test_pred)

cm_train_pct = cm_train / cm_train.astype(np.float).sum()*100
cm_val_pct = cm_val / cm_val.astype(np.float).sum()*100
cm_test_pct = cm_test / cm_test.astype(np.float).sum()*100

summary = [
    ['------', 'Train', 'Validation', 'Test'],
    ['confusion matrix', cm_train, cm_val, cm_test],
    ['confusion matrix pct', cm_train_pct, cm_val_pct, cm_test_pct],
    ['f1_score', f1_score(train_y, train_pred), f1_score(val_y, val_pred), f1_score(test_y, test_pred)],
    ['accuracy score', accuracy_score(train_y, train_pred), accuracy_score(val_y, val_pred), accuracy_score(test_y, test_pred)],
    ['recall score', recall_score(train_y, train_pred), recall_score(val_y, val_pred), recall_score(test_y, test_pred)],
    ['precision_score', precision_score(train_y, train_pred), precision_score(val_y, val_pred), precision_score(test_y, test_pred)]
]

# print pretty table
pretty_table(summary)

------              	Train                      	Validation                 	Test                       
confusion matrix    	[[284507   1796]           	[[80986   611]             	[[121148   1108]           
                    	 [  2005   9685]]          	 [  771  2591]]            	 [  1376   3662]]          
confusion matrix pct	[[95.47439034  0.60269872] 	[[95.32362669  0.71917042] 	[[95.171807    0.87042594] 
                    	 [ 0.6728346   3.25007634]]	 [ 0.90749656  3.04970633]]	 [ 1.08096218  2.87680488]]
f1_score            	0.8359587415303612         	0.7894576477757466         	0.7467373572593801         
accuracy score      	0.9872446668210327         	0.9837333301945644         	0.9804861187487234         
recall score        	0.8284858853721129         	0.7706722189173111         	0.7268757443429933         
precision_score     	0.8435676334814041         	0.8091817613991256         	0.7677148846960168         
